# Make plots map
Create some plots on a map.

In [67]:
using PyPlot
using NCDatasets
using PyCall
using Dates
const plt = PyPlot
ccrs = pyimport("cartopy.crs")
cfeature = pyimport("cartopy.feature")
include("./config.jl")
include("./ME4OH.jl")
mainproj = ccrs.Mollweide(central_longitude=200)
datacrs = ccrs.PlateCarree();
coast = cfeature.GSHHSFeature(scale="h")

[ Info: File already downloaded


PyObject <cartopy.feature.GSHHSFeature object at 0x77e2c7357850>

In [3]:
@info("Figures will be saved in $(figdir)")

[ Info: Figures will be saved in ../figures/


## Observations 

In [5]:
thetimeperiod = timeperiod2
datadirdisk = joinpath(databasedir, "$(thetimeperiod[1])-$(thetimeperiod[end])")

"/media/ctroupin/T7 Shield/000060826v009/data/en4.1.1/2005-2014"

In [8]:
datafilelist = ME4OH.get_filelist(datadirdisk)
datafile = datafilelist[1]

[ Info: Found 120 files


"/media/ctroupin/T7 Shield/000060826v009/data/en4.1.1/2005-2014/ofam3-jra55.all.EN.4.1.1.f.profiles.g10.200501.update.extra.danom.200501_201412.nc"

In [63]:
for indexlayer = 1:3
    lon, lat, dates, dohc, adohc, dadohc, dohc_mask, bounds, depth_level_thickness = ME4OH.read_profile(datafile);
    goodvalues = goodvalues = .!(isnan.(dohc[indexlayer,:]));
    
    fig = plt.figure()
    ax = plt.subplot(111, projection=mainproj)
    ax.set_global()
    scat = ax.scatter(lon, lat, s=3, c=dohc[indexlayer,:], transform=datacrs)
    cb = plt.colorbar(scat, shrink=.6)
    cb.set_label("TJ/m²", rotation=0, ha="left")
    ax.set_title("Density of Ocean Heat Content\n $(Dates.monthname(Dates.month(dates[1]))) $(Dates.year(dates[1])), layer [$(depthlayers[indexlayer][1])-$(depthlayers[indexlayer][2])] m")
    ax.add_feature(coast, lw=.5, zorder=4)
    fname = "dohc_$(Dates.year(dates[1]))$(lpad(Dates.month(dates[1]), 2, '0'))_layer$(indexlayer)"
    plt.savefig(joinpath(figdir, fname))
    plt.close()
end    

## Gridded fields

In [71]:
outputfile = joinpath(mainoutputdir, "experiment-A/dohc/OHC_1979_2014_lev0_286.6_expA_DIVAnd.nc")

"../output/experiment-A/dohc/OHC_1979_2014_lev0_286.6_expA_DIVAnd.nc"

In [72]:
nc = NCDataset(outputfile, "r")
dates = nc["time"][:]
dohc = nc["dohc"][:,:,:]
close(nc)

closed Dataset

In [75]:
timeindex = findfirst(dates .== DateTime(2005, 1, 1))

313

In [82]:
fig = plt.figure()
ax = plt.subplot(111, projection=mainproj)
ax.set_global()
pcm = ax.pcolormesh(longrid, latgrid, dohc[:,:,timeindex]', transform=datacrs)
cb = plt.colorbar(pcm, shrink=.6)
cb.set_label("TJ/m²", rotation=0, ha="left")
ax.set_title("Interpolated density of Ocean Heat Content\nExperiment A\n $(Dates.monthname(Dates.month(dates[1]))) $(Dates.year(dates[1])), layer [$(depthlayers[indexlayer][1])-$(depthlayers[indexlayer][2])] m")
ax.add_feature(coast, lw=.5, zorder=4)
fname = "dohc_expA_$(Dates.year(dates[1]))$(lpad(Dates.month(dates[1]), 2, '0'))_layer$(indexlayer)"
plt.savefig(joinpath(figdir, fname))
plt.close()
plt.show() 